In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import html5lib

In [2]:
def makeSoup(url):
    response=requests.get(url)
    soup=BeautifulSoup(response.content,"lxml")
    return soup

The code below is a way to try and find the special characteristics of the episode. Check back later on how this works.

In [3]:
def findBetween( s, first, last ):
    try:
        start = s.index( first ) + len( first )
        end = s.index( last, start )
        return s[start:end]
    except ValueError:
        return ""

def findBetweenR( s, first, last ):
    try:
        start = s.rindex( first ) + len( first )
        end = s.rindex( last, start )
        return s[start:end]
    except ValueError:
        return ""

In [4]:
def removePrefix(text, prefix):
    return text[len(prefix):] if text.startswith(prefix) else text
        

In [120]:
bachelorette_seas = list(np.arange(4,13))

In [148]:
soup = makeSoup('https://en.wikipedia.org/wiki/The_Bachelorette_(season_7)')

In [124]:
def soupCleanser(soup):
    import re
    import numpy as np
    import pandas as pd
    # make the soup
    
    # find the right table
    tables = soup.findChildren('table')
    table = tables[2]
    
    # find the descriptive colors in hex format
    descript_colors_cols = soup.findChildren('dd')
    styles = [color.span['style'] for color in descript_colors_cols]
    colors = [findBetween(color,"background-color:",";") for color in styles]
    
    
    messy_color_codes = [dd.text for dd in descript_colors_cols]
    color_codes = [removePrefix(code,"\xa0\xa0\xa0\xa0 ") for code in messy_color_codes]
    
    color_dict = dict(zip(colors, color_codes))

    # turn the data into a workable format
    data   = [[td.text for td in row.select('td')]
             for row in table.findAll('tr')]
    
    body = data[1:][1:]
    cols   =  zip(*body)
    
    body = [[re.sub(r'\[\w+\]', '', i) for i in row]
        for row in body]
    
    for i in body:
        if len(i) == len(body[0]):
            pass
        else:
            a = [" "]
            a.extend(" " * (len(body[0])))
            diff = len(a) - len(i) 
            n = i.extend( a[0:diff])
                
    contestants = [row[0] for row in body]
    
    # create lists of call out
    call_out_order_list = []
    for i in list(np.arange(0,(len(body[0])-1))):
        c = [row[i] for row in body] # column
        ci = list(np.arange(1,len(c) + 1)) # column index/counter
        cdict = dict(zip(c, ci)) # creaet dictionary
        col_out_col = [cdict.get(k, 'eliminated') for k in contestants] # call out order
        call_out_order_list.append(col_out_col)
    
    call_out_order_list[0] = contestants # replace first row with contestant names
    
    # find headers for data frame creation
    d = [[td.text for td in row.select('th')]
        for row in table.findAll('tr')]
    
    season_headers = d[1]
    header = ["contestants"] + season_headers
    
    header = [re.sub(r'\[\w+\]', '', i) for i in header]
    
    vov = dict(zip(header, call_out_order_list))
    
    return pd.DataFrame(vov)

In [169]:
soupCleanser(soup)

,1,2,3,4,5,6,7,8,9,contestants
0,13,eliminated,eliminated,eliminated,eliminated,eliminated,eliminated,eliminated,eliminated,Chris H.
1,3,3,4,eliminated,eliminated,eliminated,eliminated,eliminated,eliminated,Jesse
2,9,7,3,1,5,4,1,2,2,Chris L.
3,4,2,6,8,2,1,5,eliminated,eliminated,Ty
4,7,1,9,3,4,2,2,3,3,Frank
5,2,8,11,7,3,7,eliminated,eliminated,eliminated,Justin
6,eliminated,eliminated,eliminated,eliminated,eliminated,eliminated,eliminated,eliminated,eliminated,Jay
7,12,13,5,5,8,eliminated,eliminated,eliminated,eliminated,Chris N.
8,17,4,7,9,9,eliminated,eliminated,eliminated,eliminated,Kasey
9,eliminated,eliminated,eliminated,eliminated,eliminated,eliminated,eliminated,eliminated,eliminated,Kyle


In [126]:
frames = []
for i in bachelorette_seas:
    soup = makeSoup('https://en.wikipedia.org/wiki/The_Bachelorette_(season_{})'.format(i))
    frame = soupCleanser(soup)
    frames.append(frame)

In [128]:
frames

[             1           2           3           4           5           6  \
 0           15           7           7           7  eliminated  eliminated   
 1           11           1  eliminated  eliminated  eliminated  eliminated   
 2            5           2           8           6           3           4   
 3            8          10           9           2  eliminated  eliminated   
 4            3          11          12  eliminated  eliminated  eliminated   
 5           14           8           2           3           2           3   
 6   eliminated  eliminated  eliminated  eliminated  eliminated  eliminated   
 7            2           5           5           1           4           1   
 8   eliminated  eliminated  eliminated  eliminated  eliminated  eliminated   
 9           10  eliminated  eliminated  eliminated  eliminated  eliminated   
 10  eliminated  eliminated  eliminated  eliminated  eliminated  eliminated   
 11  eliminated  eliminated  eliminated  eliminated 

In [168]:



# read it back in
with open("/Users/jamesbain/Desktop/bachelor/data/raw/bachelorette6.html", "rb") as f:
    soup = BeautifulSoup(f, "lxml")
    
soup

<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="en">
<head>
<meta charset="utf-8"/>
<title>The Bachelorette (season 6) - Wikipedia</title>
<script>document.documentElement.className = document.documentElement.className.replace( /(^|\s)client-nojs(\s|$)/, "$1client-js$2" );</script>
<script>(window.RLQ=window.RLQ||[]).push(function(){mw.config.set({"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":false,"wgNamespaceNumber":0,"wgPageName":"The_Bachelorette_(season_6)","wgTitle":"The Bachelorette (season 6)","wgCurRevisionId":745970469,"wgRevisionId":745970469,"wgArticleId":27336922,"wgIsArticle":true,"wgIsRedirect":false,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Articles to be expanded from December 2011","All articles to be expanded","Articles using small message boxes","American dating and relationship reality television series","2010 American television seasons"],"wgBreakFrames":false,"wgPageContentLanguage":"en","wgPageContentModel":"wik

In [165]:
bachelorette_seas = list(np.arange(4,13))

urls = ['https://en.wikipedia.org/wiki/The_Bachelorette_(season_{})'.format(i) for i in bachelorette_seas]

name = requests.get(urls[0])

with open("/Users/jamesbain/Desktop/bachelor/data/raw/bachelorette4.html", "wb") as f:
    f.write(name.content)
    
name = requests.get(urls[1])

with open("/Users/jamesbain/Desktop/bachelor/data/raw/bachelorette5.html", "wb") as f:
    f.write(name.content)
    
name = requests.get(urls[2])

with open("/Users/jamesbain/Desktop/bachelor/data/raw/bachelorette6.html", "wb") as f:
    f.write(name.content)
    

name = requests.get(urls[3])

with open("/Users/jamesbain/Desktop/bachelor/data/raw/bachelorette7.html", "wb") as f:
    f.write(name.content)
    
name = requests.get(urls[4])

with open("/Users/jamesbain/Desktop/bachelor/data/raw/bachelorette8.html", "wb") as f:
    f.write(name.content)
    
name = requests.get(urls[5])

with open("/Users/jamesbain/Desktop/bachelor/data/raw/bachelorette9.html", "wb") as f:
    f.write(name.content)

name = requests.get(urls[6])

with open("/Users/jamesbain/Desktop/bachelor/data/raw/bachelorette10.html", "wb") as f:
    f.write(name.content)
    
name = requests.get(urls[7])

with open("/Users/jamesbain/Desktop/bachelor/data/raw/bachelorette11.html", "wb") as f:
    f.write(name.content)
    
name = requests.get(urls[8])

with open("/Users/jamesbain/Desktop/bachelor/data/raw/bachelorette12.html", "wb") as f:
    f.write(name.content)

# old code

In [69]:
#def dataCollector(soup):
#    import re
#    import numpy as np
#    # make the soup
#    this = makeSoup(soup)
#    
#    # find the right table
#    tables = this.findChildren('table')
#    table = tables[2]
#    
#    
#    # find the descriptive colors in hex format
#    descript_colors_cols = this.findChildren('dd')
#    styles = [color.span['style'] for color in descript_colors_cols]
#    colors = [findBetween(color,"background-color:",";") for color in styles]
#    
#    
#    messy_color_codes = [dd.text for dd in descript_colors_cols]
#    color_codes = [removePrefix(code,"\xa0\xa0\xa0\xa0 ") for code in messy_color_codes]
#    
#    color_dict = dict(zip(colors, color_codes))
#
#    # turn the data into a workable format
#    data   = [[td.text for td in row.select('td')]
#             for row in table.findAll('tr')]
#    
#    body = data[1:][1:]
#    cols   =  zip(*body)
#    
#    body_n = []
#
#    for i in body:
#        if len(i) == len(body[0]):
#            body_n.append(i)
#        else:
#            a = [" "]
#            a.extend(" " * (len(body[0])))
#            diff = len(a) - len(i) 
#            n = i.extend( a[0:diff])
#            body_n.append(n)
#                
#    # create a dict with the data
#    #tbl_d  = {name:col for name, col in zip(header,cols)}
#    contestants = [row[0] for row in body]
#    
#    #c2 = [row[2] for row in body]
#    #ci2 = list(np.arange(1,len(c2) + 1))
#    #c2dict = dict(zip(c2, ci2))
#    
#    #yup = [c2dict[x] for x in contestants]
#    #yup = [c2dict.get(k, 'eliminated') for k in contestants] # default to eliminated if name doesn't exist
#    
#    elimination_list = []
#    for i in list(np.arange(0,(len(body[0])-1))):
#        c = [row[i] for row in body]
#        ci = list(np.arange(1,len(c) + 1))
#        cdict = dict(zip(c, ci))
#        elim_col = [cdict.get(k, 'eliminated') for k in contestants]
#        elimination_list.append(elim_col)
#          
#    return elimination_list

In [70]:
#body = dataCollector('https://en.wikipedia.org/wiki/The_Bachelorette_(season_4)')